
... ***CURRENTLY UNDER DEVELOPMENT*** ...


### TODO
- INTRODUCIR PLOTEO SCA_VAL
- PLOT PERIODO RETORNO ANNUAL MAXIMA
- VALIDACION VS BOYA AWAC + PLOTEO

- RECONSTRUIR LEVEL NEARSHORE!!! (en serie temporal. Tengo swells y seas swan-propagados.)

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import numpy as np
import pandas as pd
import xarray as xr

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..', '..'))

# teslakit
from teslakit.database import Database
from teslakit.rbf import RBF_Reconstruction, RBF_Validation

from teslakit.plotting.extremes import Plot_ReturnPeriodValidation



## Database and Site parameters

In [2]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/nico/Projects/TESLA-kit/TeslaKit/data'
db = Database(p_data)

# set site
db.SetSite('ROI')


In [3]:
# input files: dataset, subset (MDA classification), and target (SWAN simulations output)

# sea 
wvs_sea_dataset_sim = db.Load_NEARSHORE_SIM_sea()
wvs_sea_dataset_hist = db.Load_NEARSHORE_HIST_sea()
wvs_sea_subset = db.Load_NEARSHORE_MDA_sea()
wvs_sea_target = db.Load_NEARSHORE_TARGET_sea()

# swells 
wvs_swell_dataset_sim = db.Load_NEARSHORE_SIM_swell()
wvs_swell_dataset_hist = db.Load_NEARSHORE_HIST_swell()
wvs_swell_subset = db.Load_NEARSHORE_MDA_swell()
wvs_swell_target = db.Load_NEARSHORE_TARGET_swell()

# keep datasets n_sim column
sea_n_sim = wvs_sea_dataset_sim['n_sim']
swl_n_sim = wvs_swell_dataset_sim['n_sim']


# remove nans (if any) from subset and target
def fix_target_nans(subset, target):
    'remove NaN data indexes from subset and target. RBF does not handle NaNs'
    
    r_nan = target.isnull().any(axis=1)  # find any row with nans

    if r_nan.any():
        # log
        print('remove nan data found at target:')
        print(target[r_nan])
        
        # clean data
        target = target[~r_nan]
        subset = subset[~r_nan]
        
    return subset, target

wvs_sea_subset, wvs_sea_target = fix_target_nans(wvs_sea_subset, wvs_sea_target)
wvs_swell_subset, wvs_swell_target = fix_target_nans(wvs_swell_subset, wvs_swell_target)


# ensure dataset and subset have same variables and column order
vns_ds = ['hs', 'tp', 'dir']

wvs_sea_dataset_sim = wvs_sea_dataset_sim[vns_ds].values
wvs_sea_dataset_hist = wvs_sea_dataset_hist[vns_ds].values
wvs_sea_subset = wvs_sea_subset[vns_ds].values

wvs_swell_dataset_sim = wvs_swell_dataset_sim[vns_ds].values
wvs_swell_dataset_hist = wvs_swell_dataset_hist[vns_ds].values
wvs_swell_subset = wvs_swell_subset[vns_ds].values


# select target variables
vns_tgt = ['Hsig', 'TPsmoo', 'Dir']

wvs_sea_target = wvs_sea_target[vns_tgt].values
wvs_swell_target = wvs_swell_target[vns_tgt].values


remove nan data found at target:
         lat       lon  Hsig  Tm02  Dir  Dspr  TPsmoo
case                                                 
49    9.4165  167.4679   0.0   NaN  NaN   NaN     NaN


In [4]:
# RBF settings

# subset - scalar / directional indexes
ix_scalar_subset = [0,1]      # scalar (hs, tp)
ix_directional_subset = [2]   # directional (dir)

# target - scalar / directional indexes
ix_scalar_target = [0,1]      # scalar (Hsig, Tpsmoo, Dir)
ix_directional_target = [2]   # directional (Dir)



## RBF Reconstruct SEA Waves

In [5]:
# --------------------------------------
# RBF Interpolation


# Simulated SEA waves
sea_rec_sim = RBF_Reconstruction(
    wvs_sea_subset, ix_scalar_subset, ix_directional_subset,
    wvs_sea_target, ix_scalar_target, ix_directional_target,
    wvs_sea_dataset_sim)
df_sea_recon_sim = pd.DataFrame(data=sea_rec_sim, columns=vns_tgt)
df_sea_recon_sim['n_sim'] = sea_n_sim  # keep n_sim value


# Historical SEA waves
sea_rec_hist = RBF_Reconstruction(
    wvs_sea_subset, ix_scalar_subset, ix_directional_subset,
    wvs_sea_target, ix_scalar_target, ix_directional_target,
    wvs_sea_dataset_hist)
df_sea_recon_hist = pd.DataFrame(data=sea_rec_hist, columns=vns_tgt)


# store data
db.Save_NEARSHORE_RECONSTRUCTION_SIM_sea(df_sea_recon_sim)
db.Save_NEARSHORE_RECONSTRUCTION_HIST_sea(df_sea_recon_hist)


ix_scalar: 0,  optimization: 47.94 | interpolation: 47.33
ix_scalar: 1,  optimization: 85.43 | interpolation: 48.12
ix_directional: 2,  optimization: 105.51 | interpolation: 100.57
ix_scalar: 0,  optimization: 54.04 | interpolation: 8.04
ix_scalar: 1,  optimization: 92.19 | interpolation: 7.85
ix_directional: 2,  optimization: 110.01 | interpolation: 21.35



## RBF Reconstruct SWELL Waves

In [7]:
# --------------------------------------
# RBF Interpolation


# Simulated SWELL waves
swl_rec_sim = RBF_Reconstruction(
    wvs_swell_subset, ix_scalar_subset, ix_directional_subset,
    wvs_swell_target, ix_scalar_target, ix_directional_target,
    wvs_swell_dataset_sim)
df_swl_recon_sim = pd.DataFrame(data=swl_rec_sim, columns=vns_tgt)
df_swl_recon_sim['n_sim'] = swl_n_sim  # keep n_sim value


# Historical SWELL waves
swl_rec_hist = RBF_Reconstruction(
    wvs_swell_subset, ix_scalar_subset, ix_directional_subset,
    wvs_swell_target, ix_scalar_target, ix_directional_target,
    wvs_swell_dataset_hist)
df_swl_recon_hist = pd.DataFrame(data=swl_rec_hist, columns=vns_tgt)


# store data
db.Save_NEARSHORE_RECONSTRUCTION_SIM_swell(df_swl_recon_sim)
db.Save_NEARSHORE_RECONSTRUCTION_HIST_swell(df_swl_recon_hist)


ix_scalar: 0,  optimization: 48.51 | interpolation: 310.64
ix_scalar: 1,  optimization: 139.96 | interpolation: 363.93
ix_directional: 2,  optimization: 149.62 | interpolation: 729.73
ix_scalar: 0,  optimization: 48.81 | interpolation: 29.58
ix_scalar: 1,  optimization: 94.31 | interpolation: 30.07
ix_directional: 2,  optimization: 97.98 | interpolation: 61.31


## Methodology Validation: Buoy comparison

In [9]:
# TODO: validate with AWAC + plot


## Methodology Validation: Annual Maxima

In [12]:
# SEA

print(df_sea_recon_sim)

# TODO: add time para el plot
# TODO: xr.from_dataframe() + n_sim index

import sys; sys.exit()

vns = ['Hsig', 'Tpsmo']
for vn in vns:

    # calculate Annual Maxima values for historical and simulated data
    hist_A = WVS_hist[vn].groupby('time.year').max(dim='time')
    sim_A = xr.concat([x[vn].groupby('time.year').max(dim='time') for x in l_WVS_sim_a], 'n_sim')

    # Return Period historical vs. simulations
    Plot_ReturnPeriodValidation(hist_A, sim_A);
    

            Hsig    TPsmoo         Dir  n_sim
0       0.208115  2.429508  101.707861    NaN
1       0.265490  2.606427   92.338007    0.0
2       0.290903  2.989002  114.112774    NaN
3       0.573906  5.413579  112.642724    NaN
4       0.347916  3.074125  106.282561    NaN
...          ...       ...         ...    ...
389576  0.288512  2.783956  104.232569    NaN
389577  0.216576  2.300218   83.985624    NaN
389578  0.349989  2.931648   93.273763    3.0
389579  0.669135  9.223054  108.896141    3.0
389580  0.271511  2.646362   96.297690    3.0

[389581 rows x 4 columns]


SystemExit: 